In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint as pp
p = print

BASE_PATH = '../../Raw data/'

Reading of DF users ed educational facilities

In [2]:
users = pd.read_csv(f'{BASE_PATH}/users.csv')
edu_fac = pd.read_csv(f'{BASE_PATH}/edu/educational_facilities.csv')
users.set_index('id', inplace=True)
edu_fac.set_index('id', inplace=True)

In [3]:
#users
#edu_fac

Function for convert an address to coordinates: latitude e longitude 

In [4]:
def get_coordinates(address):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": address,
        "format": "json"
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        first_result = data[0]
        latitude = first_result["lat"]
        longitude = first_result["lon"]
        return latitude, longitude
    else:
        return None, None

Function for create DF places with the right ID where df_name -> name of our DF users and edu_fac; location_columns -> depends by the name of the column where there is the address; id_prefix -> u = user and e = educational facility + also use the function for the coordinates 

In [5]:
def create_places(df_name, location_columns, id_prefix):
    places_infos = []

    for _, row in df_name.iterrows():
        for location_type in location_columns:
            location_value = row[location_type]
            places_infos.append({'address': location_value})

    places_df = pd.DataFrame(places_infos, columns=['id', 'address'])
    places_df.drop_duplicates(subset='address', inplace=True)
    places_df.reset_index(drop=True, inplace=True)
    places_df['id'] = id_prefix + (places_df.index + 1).astype(str)
    places_df[['latitude', 'longitude']] = places_df['address'].apply(get_coordinates).apply(pd.Series)

    return places_df


Use the function for create DF places to create the DF places_users with the right column ID 

In [6]:
u_locations = ['domicile_location', 'residence_location', 'work_location']
places_users = create_places(users, u_locations, 'u')
places_users

,id,address,latitude,longitude
0,u1,Via della Lovara 9 38012 Mollaro TN,46.296044,11.0736456
1,u2,Via Alpini 17 38017 Mezzolombardo TN,46.2117788,11.0959909
2,u3,Via alla Cascata 2 38123 Povo TN,46.0664712,11.1531061
3,u4,Via Zambra 3 38121 Trento TN,46.09307695,11.116801775933974
4,u5,Via Luigi Einaudi 60 38122 Trento TN,46.0524427,11.1237744
5,u6,Via Cristoforo Madruzzo 24 38122 Trento TN,46.063605,11.1203672
6,u7,Via Giuseppe Mazzini 35 38122 Trento TN,46.0660016,11.1225188
7,u8,Corso Bettini 84 Rovereto 38068 TN,45.8936522,11.0435393
8,u9,Via della Malpensada 140 38123 Trento TN,46.0474887,11.1334388
9,u10,Via Giovanni Amendola 15 Bologna 40121 BO,44.5048727,11.3413194


Use the function for create DF places to create the DF places_education_facility with the right column ID 

In [7]:
edu_locations = ['address']
places_edu_fac = create_places(edu_fac, edu_locations, 'e')
places_edu_fac

,id,address,latitude,longitude
0,e1,Via Don Anzelini 2 38061 Ala TN,45.7559873,11.0039826
1,e2,Via Manzoni 10 38041 Albiano TN,46.14216115,11.194774408550122
2,e3,Via della Croce 2 38060 Aldeno TN,45.9751106,11.0949833
3,e4,Via Giuseppe Garibaldi 4 38049 Altopiano dell...,46.0041396,11.1973895
4,e5,Piazza S. Vito 1 38010 Andalo TN,None,None
...,...,...,...,...
117,e118,Via Giuseppe Verdi 26 38122 Trento,46.0662011,11.1198995
118,e119,Via Giuseppe Verdi 53 38122 Trento,46.0679235,11.1191718
119,e120,Via Edmund Mach 1 38010 San Michele All'adige TN,46.1946899,11.1346593
120,e121,via S. Maria Maddalena 1 38122 Trento,46.0685816,11.1254832


Replace in the old DF users the columns 'domicile_location', 'residence_location', 'work_location' where there were address with the corresponding ID taken from DF places_users 

In [8]:
user_id = dict(zip(places_users['address'], places_users['id']))

users['domicile_location'] = users['domicile_location'].map(user_id)
users['residence_location'] = users['residence_location'].map(user_id)
users['work_location'] = users['work_location'].map(user_id)

users

,name,age,domicile_location,residence_location,work_location,edu_type,occupation,special_needs
id,,,,,,,,
1,Mario,13,u1,u1,u2,MS,Stud.,NaN
2,Jessica,16,u3,u3,u4,HS,Stud.,NaN
3,Luca,18,u5,u5,u6,HS,Stud.,Access.
4,Gian,20,u7,u7,u8,U,Stud.,NaN
5,Gaia,22,u9,u10,u11,U,Stud.,NaN
6,Carla,22,u9,u12,u13,U,Stud.,Space
7,Fausto,32,u14,u14,u15,U,Prof.,NaN
8,Lucia,43,u16,u16,u17,U,Prof.,TimeOpt.
9,Anna,52,u18,u18,u19,MS,Janitor,NaN


Replace in the old DF edu_fac the column 'address' where there was address with the corresponding ID taken from DF places_edu_fac  

In [9]:
edu_id = dict(zip(places_edu_fac['address'], places_edu_fac['id']))

edu_fac['address'] = edu_fac['address'].map(edu_id)
edu_fac.drop_duplicates(subset='address', inplace=True)

edu_fac

,type,name,address
id,,,
TNMM84801R,middleschool,Ala Cesare Battisti,e1
TNMM817026,middleschool,Albiano,e2
TNMM82101R,middleschool,Aldeno,e3
TNMM803017,middleschool,Altopiano Vigolana-Vigolo Vatt.,e4
TNMM82403A,middleschool,Andalo Cesare Battisti,e5
...,...,...,...
10,university,Dipartimento di Sociologia e Ricerca Sociale,e118
11,university,Facoltà di Giurisprudenza,e119
12,university,Centro Agricoltura; Alimenti; Ambiente - C3A,e120


Union of the two DF places_edu_fac and places_users to obtain the final DF places with all the information of address, latitude, and longitude + the right value of the column ID corresponding with columns in DF users (u1, u2, u...) and DF edu_fac (e1, e2, e...)

In [10]:
places = pd.concat([places_edu_fac, places_users], ignore_index=True)

places

,id,address,latitude,longitude
0,e1,Via Don Anzelini 2 38061 Ala TN,45.7559873,11.0039826
1,e2,Via Manzoni 10 38041 Albiano TN,46.14216115,11.194774408550122
2,e3,Via della Croce 2 38060 Aldeno TN,45.9751106,11.0949833
3,e4,Via Giuseppe Garibaldi 4 38049 Altopiano dell...,46.0041396,11.1973895
4,e5,Piazza S. Vito 1 38010 Andalo TN,None,None
...,...,...,...,...
136,u15,Via Sommarive 14 38123 Povo TN,46.0652663,11.1505456
137,u16,Case Ferrovieri Porta Nuova 37138 Verona VR,45.4303186,10.979640081121133
138,u17,Via Vigilio Inama 5 Trento 38122 TN,46.0662413,11.1183642
139,u18,Via Gustavo Modena 32 38065 Mori TN,45.8529104,10.9787196


CSV extraction of single DF 

In [11]:
places.to_csv(f'{BASE_PATH}/places_final/places_final.csv', index=False)
users.to_csv(f'{BASE_PATH}/places_final/users_final.csv', index=False)
edu_fac.to_csv(f'{BASE_PATH}/places_final/edu_fac_final.csv', index=False) 